# Bias Correction

Caleb Phillips (caleb.phillips@nrel.gov), Lindsay Sheridan (lindsay.sheridan@pnnl.gov), Dmitry Duplyakin (dmitry.duplyakin@nrel.gov), and Jenna Ruzekowicz (jenna.ruzekowicz@nrel.gov)

This notebook will read resource data and reference observation data and use it to compute a bias corrected version of the resource data (by multiple linear regression) for those sites where reference data have been identified.

In [ ]:
import pandas as pd
import h5pyd
from dw_tap.data_fetching import getData
from tqdm import tqdm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import plotly.express as px
import numpy as np

fetch_wtk_data = False

In [ ]:
from rex.resource_extraction import MultiYearWindX
from dw_tap.data_fetching import get_data_wtk_led_on_eagle 
from dw_tap.data_fetching import getData

In [ ]:
sites = pd.read_csv("01 Bergey Turbine Data/bergey_sites.csv")
sites.head()

### Download 2018 WTK LED Data at Each Bias Location

In [ ]:
if fetch_wtk_data:

    debug = True
    wtk_dfs = []
    files = ['/campaign/tap/CONUS/wtk/5min/2018/%s/conus_2018-%s.h5' % \
             (str(i).zfill(2), str(i).zfill(2)) for i in range(1,13)]
    for row in tqdm(sites.to_dict(orient="records")):
        atmospheric_df = pd.DataFrame()
        tid = row['APRS ID']
        lat = row['Met Tower Latitude']
        lon = row['Met Tower Longitude']
        heights = str(row['Measurement Height (m)'])
        if np.isnan(lat) or np.isnan(lon):
            continue
        for h in heights.split(","):
            h = int(float(h))
            if(debug):
                print("Fetching data for turbine %s (%f,%f) at height %d" % (tid,lat,lon,h))
            for file in files:
                myr = MultiYearWindX(file, hsds=False)
                d = get_data_wtk_led_on_eagle(myr, 
                                              lat, lon, h, "IDW", 
                                              power_estimate=False,
                                              start_time_idx=None, 
                                              end_time_idx=None,
                                              time_stride=None)
                atmospheric_df = pd.concat([atmospheric_df, d])

            atmospheric_df['tid'] = tid
            atmospheric_df['h'] = h

            wtk_dfs.append(atmospheric_df)
            
    wtk_df_2018 = pd.concat(wtk_dfs)
    wtk_df_2018.head()
    wtk_df_2018['datetime'] = pd.to_datetime(wtk_df_2018["datetime"]).dt.tz_convert('UTC')
    wtk_df_2018.to_csv("02 Bias Correction/wtk_led_2018_met.csv.bz2",index=False)

### Download 2019 WTK LED Data at Each Bias Location

In [ ]:
if fetch_wtk_data:
    wtk_dfs = []

    # 2019 hourly file
    myr = MultiYearWindX('/campaign/tap/CONUS/wtk/60min/2019/conus_2019.h5', hsds=False)

    for row in tqdm(sites.to_dict(orient="records")):
        tid = row['APRS ID']
        lat = row['Met Tower Latitude']
        lon = row['Met Tower Longitude']
        heights = str(row['Measurement Height (m)'])
        if np.isnan(lat) or np.isnan(lon):
            continue
        for h in heights.split(","):
            h = int(float(h))
            if(debug):
                print("Fetching data for turbine %s (%f,%f) at height %d" % (tid,lat,lon,h))
            atmospheric_df = pd.DataFrame()

            atmospheric_df = get_data_wtk_led_on_eagle(myr, 
                                              lat, lon, h, "IDW", 
                                              power_estimate=False,
                                              start_time_idx=None, 
                                              end_time_idx=None,
                                              time_stride=None)

            atmospheric_df['tid'] = tid
            atmospheric_df['h'] = h
            wtk_dfs.append(atmospheric_df)
            
    wtk_df_2019 = pd.concat(wtk_dfs)
    wtk_df_2019.head()
    wtk_df_2019['datetime'] = pd.to_datetime(wtk_df_2019["datetime"]).dt.tz_convert('UTC')
    wtk_df_2019.to_csv("02 Bias Correction/wtk_led_2019_met.csv.bz2",index=False)

### Combine 2018 and 2019 WTK Data

In [ ]:
if not fetch_wtk_data:
    wtk_df_2019 = pd.read_csv("02 Bias Correction/wtk_led_2019_met.csv.bz2")
    wtk_df_2018 = pd.read_csv("02 Bias Correction/wtk_led_2018_met.csv.bz2")

wtk_dfs = pd.concat([wtk_df_2018,wtk_df_2019])
wtk_dfs['datetime'] = pd.to_datetime(wtk_dfs["datetime"]).dt.tz_convert('UTC')

In [ ]:
wtk_dfs.head()

In [ ]:
wtk_dfs.dtypes

### Read in the met tower data, align with WTK and fit models - example of t034

Note that the below is needlessly verbose, repeating code for each site and would be much cleaner in a loop. I've done it this way so we can look at the fit/plots for each site, but may clean up in the future.

In [ ]:
bc_dfs = [] # dataframe to hold bias corrected data for each site

In [ ]:
def prepare_dataframe(tid,wtk_dfs):
    minfo = sites[sites['APRS ID'] == tid].to_dict(orient='records').pop()
    mfile = minfo["Met Tower"]
    mheight = int(minfo["Measurement Height (m)"])
    mdf = pd.read_csv("02 Bias Correction/%s" % mfile)
    mdf = mdf.rename(columns={'Time': 'datetime', "Spd%dm" % mheight: 'ws_obs', "Dir%dm" % mheight: 'wd_obs'})
    mdf['datetime'] = pd.to_datetime(mdf['datetime']).dt.tz_localize('UTC')
    print("Met data runs from %s to %s" % (mdf['datetime'].min(),mdf['datetime'].max()))
    mdf = mdf.merge(wtk_dfs[wtk_dfs['tid'] == tid],on='datetime',how='left').dropna()
    mdf['hour'] = mdf['datetime'].dt.hour
    mdf['month'] = mdf['datetime'].dt.month
    return mdf

In [ ]:
mdf = prepare_dataframe('t034',wtk_dfs)
mdf.head()

In [ ]:
mod = sm.OLS(mdf["ws_obs"],sm.add_constant(mdf[["ws","wd","hour","month"]]))
res = mod.fit()
print(res.summary())

In [ ]:
# NNLS version requires Sklearn because statsmodels doesn't have NNLS
def regression_results(y_true, y_pred):

    # Regression metrics
    #explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    #mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    #print('explained_variance: ', round(explained_variance,4))    
    #print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

fit = LinearRegression().fit(mdf[["ws","hour","month","wd"]],mdf["ws_obs"])
regression_results(fit.predict(mdf[["ws","hour","month","wd"]]),mdf["ws_obs"])

There may be some value in exploring nonlinear models (MARS, RF etc.)

In [ ]:
def plot_bc_pointcloud(mdf):
    fig = px.scatter(mdf,x='ws', y='ws_obs',labels={"ws":"WTK Windspeed (m/s)","ws_obs":"Observed Windspeed (m/s)"})
    fig.update_xaxes(range=[0,25])
    fig.update_yaxes(range=[0,25])
    fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
    fig.show()
    
plot_bc_pointcloud(mdf)

### Apply models to WTK data at site locations using NNLS model

In [ ]:
sitewtk = [pd.read_csv("01 Bergey Turbine Data/wtk_led_2019.csv.bz2"),\
           pd.read_csv("01 Bergey Turbine Data/wtk_led_2018.csv.bz2")]
sitewtk = pd.concat(sitewtk)
sitewtk.rename(columns={"packet_date":"datetime"},inplace=True)
sitewtk.head()

In [ ]:
def do_correction(sitewtk,tid,fit):
    chunk = sitewtk[sitewtk['tid'] == tid].reset_index()
    chunk['datetime'] = pd.to_datetime(chunk['datetime'])
    chunk['hour'] = chunk['datetime'].dt.hour
    chunk['month'] = chunk['datetime'].dt.month
    #chunk["ws_bc"] = res.predict(sm.add_constant(chunk[["ws","hour","month","wd"]]))
    chunk["ws_bc"] = fit.predict(chunk[["ws","hour","month","wd"]])
    chunk.loc[chunk["ws_bc"] < 0,"ws_bc"] = 0
    return chunk

chunk = do_correction(sitewtk,'t034',fit)
bc_dfs.append(chunk)
chunk.head()

In [ ]:
def plot_correction(chunk):
    fig = px.scatter(chunk,x='ws', y='ws_bc',color="wd",labels={'ws':"WTK Windspeed (mps)",'ws_bc':"Bias-Corrected Winspeed"})
    fig.update_xaxes(range=[0,22])
    fig.update_yaxes(range=[0,22])
    fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
    fig.show()
    
plot_correction(chunk)

### Site 83 (California)

In [ ]:
mdf = prepare_dataframe('t083',wtk_dfs)
mdf.head()

In [ ]:
fit = LinearRegression().fit(mdf[["ws","hour","month","wd"]],mdf["ws_obs"])
regression_results(fit.predict(mdf[["ws","hour","month","wd"]]),mdf["ws_obs"])

In [ ]:
plot_bc_pointcloud(mdf)

In [ ]:
chunk = do_correction(sitewtk,'t133',fit)
bc_dfs.append(chunk)
chunk.head()

In [ ]:
plot_correction(chunk)

### Site 133 (Illinois)

No overlap with WTK-LED, skipping

In [ ]:
mdf = prepare_dataframe('t133',wtk_dfs)
mdf.head()

### Site 140 (New York)

No overlap with WTK-LED, skipping

In [ ]:
mdf = prepare_dataframe('t140',wtk_dfs)
mdf.head()

### Site 170 (Ohio)

In [ ]:
mdf = prepare_dataframe('t170',wtk_dfs)
mdf.head()

In [ ]:
fit = LinearRegression().fit(mdf[["ws","hour","month","wd"]],mdf["ws_obs"])
regression_results(fit.predict(mdf[["ws","hour","month","wd"]]),mdf["ws_obs"])

In [ ]:
plot_bc_pointcloud(mdf)

In [ ]:
chunk = do_correction(sitewtk,'t170',fit)
bc_dfs.append(chunk)
chunk.head()

In [ ]:
plot_correction(chunk)

### Site 183 (Illinois)

No overlap with WTK-LED, skipping

In [ ]:
mdf = prepare_dataframe('t183',wtk_dfs)
mdf.head()

### Site 192 (Vermont)

No overlap with WTK-LED, skipping

In [ ]:
mdf = prepare_dataframe('t192',wtk_dfs)
mdf.head()

### Site 207 (Illinois)

No overlap with WTK-LED, skipping

In [ ]:
mdf = prepare_dataframe('t207',wtk_dfs)
mdf.head()

## Save Bias Corrected Version

In [ ]:
bcdf = pd.concat(bc_dfs)
bcdf.head()

In [ ]:
bcdf.to_csv("02 Bias Correction/wtk_led_bc.csv.bz2",index=False)